<a href="https://colab.research.google.com/github/joshbrx/DataScience_challenge/blob/main/Departamentos_en_la_CDMX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Pregunta 1***
El archivo ``reto_precios.csv`` contiene información sobre precios de departamentos nuevos dentro de la CDMX, junto con diversas características del inmueble.

Para esta pregunta,requerimos que nos puedas explicar los factores que influyen en el precio por metro cuadrado de cada vivienda

***Hint: Existe información externa que te podría ser útil para resolver esta pregunta*.**



---



In [379]:
import pandas as pd
import numpy as np
import keras
import re

In [380]:
url = "https://raw.githubusercontent.com/joshbrx/DataScience_challenge/main/reto_precios.csv"
df = pd.read_csv(url)
df.head()

,main_name,subtitle,link,location,price,attributes,timestamp,id,address,since,...,apartments_per_floor,disposition,parking_lots,floor_situated,orientation,num_bedrooms,department_type,m2,final_price,price_square_meter
0,Departamento - Roma Norte,Departamento en venta,https://departamento.metroscubicos.com/MLM-139...,"Sinaloa 20 , Roma Sur, Roma Sur, Cuauhtémoc, D...",9000000 pesos,"127 m² construidos,2 recs.",2022-03-29,1,"Sinaloa 20 , Roma Sur",Publicado hace 5 días,...,NaN,Contrafrente,2,NaN,NaN,2.0,NaN,127.0,9000000.0,70866.141732
1,Preventa Exclusiva · Terraloza Roma Norte ·,Departamento en venta,https://departamento.metroscubicos.com/MLM-139...,"Tonala, Roma Norte, Cuauhtémoc, Distrito Federal",4264000 pesos,"54 m² construidos,1 rec.",2022-03-29,2,Tonala,Publicado hace 7 días,...,NaN,NaN,1,NaN,NaN,1.0,NaN,54.0,4264000.0,78962.962963
2,Townhouse Único En La Roma Sur,Departamento en venta,https://departamento.metroscubicos.com/MLM-139...,"Tepeji 43, Roma Sur, Ciudad De México, Cdmx, M...",8093500 pesos,"163 m² construidos,3 recs.",2022-03-29,3,"Tepeji 43, Roma Sur, Ciudad De México, Cdmx, M...",Publicado hace 4 días,...,NaN,NaN,2,NaN,NaN,3.0,NaN,163.0,8093500.0,49653.374233
3,"Departamento En Roma Sur, Cuauhtémoc Con 2 Rec...",Departamento en venta,https://departamento.metroscubicos.com/MLM-132...,"Linares, Roma Sur, Cuauhtémoc, Distrito Federal",4000000 pesos,"93 m² construidos,2 recs.",2022-03-29,4,Linares,Publicado hace 5 meses por {action},...,NaN,NaN,1,NaN,Oeste,2.0,Loft,93.0,4000000.0,43010.752688
4,Últimos Deptos En Preventa En Roma Norte,Departamento en venta,https://departamento.metroscubicos.com/MLM-137...,"Frontera, Roma Norte, Cuauhtémoc, Distrito Fed...",6732700 pesos,"103 m² construidos,2 recs.",2022-03-29,5,Frontera,Publicado hace 52 días,...,NaN,NaN,1,NaN,NaN,2.0,NaN,103.0,6732700.0,65366.019417


## Análisis exploratorio y limpieza en base de datos

Es necesario poder comprender el tipo de datos con el que estamos trabajando para poder realizar posteriormente una limpieza de los mismos.

In [381]:
# Se define una copia de "df" llamada "data_cdmx" para poder limpiarla y no modificar la base de datos inicial, ni perder información.
data_cdmx = df.copy()

In [382]:
df.info()
# df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   main_name             981 non-null    object 
 1   subtitle              981 non-null    object 
 2   link                  981 non-null    object 
 3   location              981 non-null    object 
 4   price                 981 non-null    object 
 5   attributes            981 non-null    object 
 6   timestamp             981 non-null    object 
 7   id                    981 non-null    int64  
 8   address               702 non-null    object 
 9   since                 981 non-null    object 
 10  description           981 non-null    object 
 11  vendor                981 non-null    object 
 12  lat                   981 non-null    float64
 13  lon                   981 non-null    float64
 14  price_mod             981 non-null    float64
 15  price_currency        9

In [383]:
df["main_name"].value_counts()

Departamento En Venta Roma Norte              84
Departamento En Venta Roma Sur                56
Departamento - Roma Norte                     49
Departamento En Venta, Roma Sur, Cdmx         14
Departamento En Venta En Roma Norte            7
                                              ..
Departamento En Venta Roma Norte (m2d1545)     1
Departamento En Venta Roma Norte (m2d1515)     1
Departamento En Venta Roma Norte (m2d1533)     1
Departamento En Venta Roma Norte (m2d1525)     1
Bajio 315 Hanover 506 (casa Roma)              1
Name: main_name, Length: 667, dtype: int64

### Subtitle (repetida)

In [384]:
# Se repite el mismo valor por lo que "subtitle" es irrelevante
print(df["subtitle"].value_counts())
data_cdmx = data_cdmx.drop("subtitle", axis=1)

Departamento en venta    981
Name: subtitle, dtype: int64


### Location

In [385]:
df["location"].value_counts()

Roma Norte, Cuauhtémoc, Distrito Federal                                  123
Roma Sur, Cuauhtémoc, Distrito Federal                                    110
Coahuila, Roma Norte, Cuauhtémoc, Distrito Federal                         70
Snsn, Roma Norte, Cuauhtémoc, Distrito Federal                             46
Departamentos Nuevos Roma Norte Coahuila, Cuauhtémoc, Distrito Federal     42
                                                                         ... 
Sinaloa, Roma Sur, Cuauhtémoc, Distrito Federal                             1
Chapultepec, Roma Norte, Cuauhtémoc, Distrito Federal                       1
Pirineos, Roma, Monterrey, Nuevo León                                       1
Hermosillo, Roma Sur, Cuauhtémoc, Distrito Federal                          1
Sinaloa 20 , Roma Sur, Roma Sur, Cuauhtémoc, Distrito Federal               1
Name: location, Length: 181, dtype: int64

### ``price`` (extracción de datos)

In [386]:
print(df["price"].value_counts())           # se trata de una variable numerica con todos sus valores enteros
data_cdmx["price"] = df['price'].apply(lambda x: int(re.search(r'\d+', x).group()))     # convertimos esta columna a datos de tipo numérico
# data_cdmx.iloc[:,[0,3]]

4495000 pesos     9
4369000 pesos     8
7040000 pesos     8
2980000 pesos     8
6790000 pesos     7
                 ..
11500000 pesos    1
9550000 pesos     1
3516884 pesos     1
7700000 pesos     1
6270000 pesos     1
Name: price, Length: 623, dtype: int64


### Attributes (doble variable y trivial)

In [387]:
# Hay 2 variables comprimidas en una sola (la primera corresponde con la columna "m2" y la segunda con "num_bedrooms")
print(df.iloc[:,5])

0      127 m² construidos,2 recs.
1        54 m² construidos,1 rec.
2      163 m² construidos,3 recs.
3       93 m² construidos,2 recs.
4      103 m² construidos,2 recs.
                  ...            
976    140 m² construidos,2 recs.
977     84 m² construidos,2 recs.
978     88 m² construidos,2 recs.
979    114 m² construidos,2 recs.
980    114 m² construidos,3 recs.
Name: attributes, Length: 981, dtype: object


In [388]:
# Verificamos los datos que no son iguales en de la columna "attribute" con la columna "m2"
df[df['attributes'].apply(lambda x: float(re.findall(r'\d+', x)[0])) != df['m2']].iloc[:,[0,5,32]].head()

,main_name,attributes,m2
189,"Preventa De Departamentos, Locales Y Oficinas ...","103 m² construidos,3 recs.",103.10
200,Venta Hermoso Depto Col. Roma Sur Baja California,"95 m² construidos,2 recs.",95.20
205,Departamentos Colonia Roma,"53 m² construidos,1 rec.",53.02
218,Departamento En Venta Colonia Roma Sur,"120 m² construidos,2 recs.",120.24
281,"Departamento En Venta Estilo Ecléctico, Roma N...","96 m² construidos,2 recs.",96.50


In [389]:
# Verificamos los casos si los datos no tienen el comportamiento esperado a pares en la columna "attributes" están bien definidos
def tmp_func1(x):
    if len(re.findall(r'\d+', x))==2:
        s = re.findall(r'\d+', x)[1]
    else: s = "turip"
    return s
df[df['attributes'].apply(tmp_func1) == "turip"].iloc[:,[0,5,30]]

,main_name,attributes,num_bedrooms
411,Venta Departamento Nuevo Colonia Roma,141 m² construidos,2.0
541,Departamento En Venta Roma Sur,"7,210 m² construidos,2 recs.",2.0
584,Preventa En La Colonia Roma,80 m² construidos,2.0
601,Preventa De Departamentos En Roma Sur,64 m² construidos,2.0
619,"Preventa En La Colonia Roma Norte, Chapultepec",34 m² construidos,2.0
631,Excelente Pre Venta En Roma Sur!,100 m² construidos,2.0
717,Excelentes Departamentos En Pre Venta En Roma ...,93 m² construidos,2.0
788,"Departamento En Venta, Col. Roma Norte",153 m² construidos,2.0
789,424587 Departamento En Venta En Col. Roma En ...,40 m² construidos,2.0
915,Preventa Departamento Zona Tec Mst,118 m² construidos,2.0


Dado que se repiten los datos de la columna ``attributes`` e incluso se encuentran mejor descritos en las columnas ``m2`` y ``num_bedrooms``, se elimina esta columna en ``data_cdmx``.

In [390]:
data_cdmx = data_cdmx.drop("attributes", axis=1);

### Timestamp (repetida)

In [391]:
df.iloc[:,6].value_counts()
data_cdmx = data_cdmx.drop("timestamp", axis=1)

### ``since`` contra ``days_on_site`` y ``since_value`` (extracción de datos)

In [392]:
df.iloc[:, 10:19].head()
df.iloc[:, [0, 9, 17, 18]].head()


,main_name,since,since_value,days_on_site
0,Departamento - Roma Norte,Publicado hace 5 días,5,5.0
1,Preventa Exclusiva · Terraloza Roma Norte ·,Publicado hace 7 días,7,7.0
2,Townhouse Único En La Roma Sur,Publicado hace 4 días,4,4.0
3,"Departamento En Roma Sur, Cuauhtémoc Con 2 Rec...",Publicado hace 5 meses por {action},5,150.0
4,Últimos Deptos En Preventa En Roma Norte,Publicado hace 52 días,52,52.0


In [393]:
# Refinamos el calculo de los días totales para medirlo en función de un mes y año promedio
def tmp_func2(x):
    tmp_var = int(re.findall('\d+', x)[0])
    if re.findall("meses", x) == ['meses']:
        tmp_var *= 30.42
    if re.findall("años", x) == ["años"]:
        tmp_var *= 365.24
    return int(round(tmp_var))

data_cdmx['since'] = df['since'].apply(tmp_func2)
data_cdmx[["main_name", "since", "days_on_site"]].head()

,main_name,since,days_on_site
0,Departamento - Roma Norte,5,5.0
1,Preventa Exclusiva · Terraloza Roma Norte ·,7,7.0
2,Townhouse Único En La Roma Sur,4,4.0
3,"Departamento En Roma Sur, Cuauhtémoc Con 2 Rec...",152,150.0
4,Últimos Deptos En Preventa En Roma Norte,52,52.0


In [394]:
# Remplazamos estos valores en "days_on_site" y eliminamos las otras variables en "data_cdmx"
data_cdmx['days_on_site'] = df['since']
data_cdmx = data_cdmx.drop(["since", "since_value"], axis=1)

### ``price_currency`` (repetida)

In [395]:
df.iloc[:,15].value_counts()
data_cdmx = data_cdmx.drop("price_currency", axis=1)

### ``amenities`` (completar valores nulos)

In [396]:
# Falta el caso para cuando no hay amenidades
df.iloc[:,19].unique()

array([ 3.,  2., nan,  4.,  5.,  6.,  7.,  8.,  1.])

In [397]:
# Rellenamos estos valores nulos con un 0
data_cdmx["amenities"] = df["amenities"].fillna(0)
data_cdmx["amenities"].unique()

array([3., 2., 0., 4., 5., 6., 7., 8., 1.])

### ``age_in_years`` (repetida)

In [398]:
df.iloc[:,20].value_counts()
data_cdmx = data_cdmx.drop("age_in_years", axis=1)

### ``bathrooms`` (dato extraño)

In [399]:
df.iloc[:,21].value_counts()

2.0     686
1.0     178
3.0     105
4.0      10
5.0       1
23.0      1
Name: bathrooms, dtype: int64

In [400]:
# Hay un dato inconsistente ya que tener 23 baños en un espacio de 91 m2 no tiene sentido
df[(df.iloc[:,21] == 23)].iloc[:, 18:]

,days_on_site,amenities,age_in_years,bathrooms,cellars,num_floors,monthly_fee,apartments_per_floor,disposition,parking_lots,floor_situated,orientation,num_bedrooms,department_type,m2,final_price,price_square_meter
687,60.0,NaN,0.0,23.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,2.0,NaN,91.0,7874000.0,86527.472527


In [401]:
# Eliminamos este dato en la tabla modificada
data_cdmx = data_cdmx.drop(687)
data_cdmx["bathrooms"].value_counts()

2.0    686
1.0    178
3.0    105
4.0     10
5.0      1
Name: bathrooms, dtype: int64

### ``cellars`` (completar valores nulos)

In [402]:
# Falta el caso para cuando no tienen almacenes/bodegas
df.iloc[:,22].unique()

array([nan,  2.,  1.])

In [403]:
# Rellenamos los valores nulos con un 0
data_cdmx["cellars"] = df["cellars"].fillna(0)
data_cdmx["cellars"].unique()

array([0., 2., 1.])

### ``monthly_fee`` sección

In [404]:
# df.iloc[:,24].value_counts()
df.iloc[:,24]

0        0 MXN
1          NaN
2          NaN
3      853 MXN
4          NaN
        ...   
976        NaN
977        NaN
978        NaN
979        NaN
980        NaN
Name: monthly_fee, Length: 981, dtype: object

In [405]:
# Primero comppletamos los valores nulos con un 0
data_cdmx["monthly_fee"] = df["monthly_fee"].fillna("0")
data_cdmx["monthly_fee"].unique()

array(['0 MXN', '0', '853 MXN', '800 MXN', '1 MXN', '3700 MXN',
       '2000 MXN', '1200 MXN', '2500 MXN', '4 MXN', '4000 MXN',
       '1500 MXN', '2900 MXN', '3900 MXN', '3000 MXN'], dtype=object)

In [406]:
data_cdmx['monthly_fee'] = data_cdmx['monthly_fee'].apply(lambda x: int(re.search(r'\d+', x).group()))
data_cdmx['monthly_fee'].unique()

array([   0,  853,  800,    1, 3700, 2000, 1200, 2500,    4, 4000, 1500,
       2900, 3900, 3000])

### ``apartments_per_floor`` (completar valores nulos)

In [413]:
# Falta el caso para cuando solo hay un apartamento por piso
df.iloc[:,25].unique()

array([nan,  2.,  7.,  3.,  4.,  1.,  5.])

In [414]:
# Rellenamos los valores nulos con un 1
data_cdmx["apartments_per_floor"] = df["apartments_per_floor"].fillna(1)
data_cdmx["apartments_per_floor"].unique()

array([1., 2., 7., 3., 4., 5.])

In [419]:
df.iloc[:,26].value_counts()

Frente          12
Interno          5
Contrafrente     3
Name: disposition, dtype: int64

In [407]:
# df.iloc[:,9].value_counts()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   main_name             981 non-null    object 
 1   subtitle              981 non-null    object 
 2   link                  981 non-null    object 
 3   location              981 non-null    object 
 4   price                 981 non-null    object 
 5   attributes            981 non-null    object 
 6   timestamp             981 non-null    object 
 7   id                    981 non-null    int64  
 8   address               702 non-null    object 
 9   since                 981 non-null    object 
 10  description           981 non-null    object 
 11  vendor                981 non-null    object 
 12  lat                   981 non-null    float64
 13  lon                   981 non-null    float64
 14  price_mod             981 non-null    float64
 15  price_currency        9